In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

### Objective of the Problem

This problem is designed for people who want to solve binary classification using Python. By the end of this notebook, you will have the necessary skills and techniques required to solve such problems. This notebook provides you with sufficient theory and practice materials to hone your skills.

### Expectation from the Notebook
This notebook is divided into the following sections:



<ul> 1).Introduction to the problem</ul>
<ul> 2).Exploratory Data Analysis (EDA) and PreProcessing</ul>
<ul> 3).Model building and Feature engineering</ul>

### Let’s look at the steps that we will follow in this notebook.

<ul> 1).Problem Statement</ul>
<ul> 2).Hypothesis Generation</ul>
<ul> 3). Getting the system ready and loading the data</ul>
<ul> 4).Understanding the data</ul>
<ul> 5).Exploratory Data Analysis (EDA)
    
    
    <ul> i).Univariate Analysis</ul>
    <ul> ii).Bivariate Analysis</ul></ul>
    
<ul> 6).Missing value and outlier treatment</ul>
<ul> 7).Evaluation Metrics for classification problems</ul>
<ul> 8).Model Building : Part I</ul>
<ul> 9).Logistic Regression using stratified k-folds cross validation</ul>
<ul> 10).Feature Engineering</ul>
<ul> 11).Model Building : Part II
    
    <ul> 1).Logistic Regression</ul>
    <ul> 2).Decision tree</ul>
    <ul> 3).Random Forest</ul>
    <ul> 4).XGBoost</ul>
</ul>

Lets look at each section in detail.

Understanding the problem statement is the first and foremost step. This would help you give an intuition of what you will face ahead of time. Let us see the problem statement -

#### Dream Housing Finance company deals in all home loans. They have presence across all urban, semi urban and rural areas. Customer first apply for home loan after that company validates the customer eligibility for loan. Company wants to automate the loan eligibility process (real time) based on customer detail provided while filling online application form. These details are Gender, Marital Status, Education, Number of Dependents, Income, Loan Amount, Credit History and others. To automate this process, they have given a problem to identify the customers segments, those are eligible for loan amount so that they can specifically target these customers.

It is a classification problem where we have to predict whether a loan would be approved or not. In a classification problem, we have to predict discrete values based on a given set of independent variable(s). Classification can be of two types:
<ul>
    <li><b>Binary Classification</b> : In this classification we have to predict either of the two given classes. For example: classifying the gender as male or female, predicting the result as win or loss, etc.</li>
    <li><b>Multiclass Classification</b> : Here we have to classify the data into three or more classes. For example: classifying a movie's genre as comedy, action or romantic, classify fruits as oranges, apples, or pears, etc.</li>
</ul>
Loan prediction is a very common real-life problem that each retail bank faces atleast once in its lifetime. If done correctly, it can save a lot of man hours at the end of a retail bank.

Although this notebook is specifically built to give you a walkthrough of Loan Prediction problem, you can always refer the content to get a comprehensive overview to solve a classification problem.

After looking at the problem statement, we will now move into hypothesis generation. It is the process of listing out all the possible factors that can affect the outcome.

### What is hypothesis generation? 
This is a very important stage in any data science/machine learning pipeline. It involves understanding the problem in detail by brainstorming as many factors as possible which can impact the outcome. It is done by understanding the problem statement thoroughly and before looking at the data.

Below are some of the factors which I think can affect the Loan Approval (dependent variable for this loan prediction problem):
<ul><li><b>Salary</b>: Applicants with high income should have more chances of loan approval.</li>
<li><b>Previous history</b>: Applicants who have repayed their previous debts should have higher chances of loan approval.</li>
<li><b>Loan amount</b>: Loan approval should also depend on the loan amount. If the loan amount is less, chances of loan approval should be high.</li>
<li><b>Loan term</b>: Loan for less time period and less amount should have higher chances of approval.</li>
<li><b>EMI</b>: Lesser the amount to be paid monthly to repay the loan, higher the chances of loan approval.</li></ul>

These are some of the factors which i think can affect the target variable, you can come up with many more factors.

### Loading Packages

In [ ]:
import seaborn as sns                  # For data visualization 
import matplotlib.pyplot as plt        # For plotting graphs 
%matplotlib inline 
import warnings   # To ignore any warnings 
warnings.filterwarnings("ignore")

### Data

For this practice problem, we have been given two CSV files: train, test .

<li>Train file will be used for training the model, i.e. our model will learn from this file. It contains all the feature variables and the target variable.</li>

<li>Test file contains all the feature variables, but not the target variable. We will apply the model to predict the target variable for the test data.</li>

### Reading data

In [ ]:
train = pd.read_csv('../input/bank-loan2/madfhantr.csv')
test = pd.read_csv('../input/bank-loan2/madhante.csv')

In [ ]:
train.head()

In [ ]:
test.head()

Let’s make a copy of train and test data so that even if we have to make any changes in these datasets we would not lose the original datasets.

In [ ]:
train_original=train.copy() 
test_original=test.copy()

In this section, we will look at the structure of the train and test datasets. Firstly, we will check the features present in our data and then we will look at their data types.

In [ ]:
train.columns

We have 12 independent variables and 1 target variable, i.e. Loan_Status in the train dataset. Let’s also have a look at the columns of test dataset.

In [ ]:
test.columns

We have similar features in the test dataset as the train dataset except the Loan_Status. We will predict the Loan_Status using the model built using the train data.

Given below is the description for each variable.

| Variables        | Description                                  |
| ---------------- | -------------------------------------------  |
| Loan_ID          | Unique Loan ID                               |
| Gender           | Male/Female                                  |
| Married          | Applicant Married (Y/N)                      |
| Dependents       | Number of dependents                         |
| Education        | Applicant Education (Graduate/Under Graduate)|
| Self_Employed    | Self employed (Y/N)                          |
| ApplicantIncome  | Applicant income                             |
| CoapplicantIncome| Co-applicant Income                          |
| LoanAmount       | Loan amount in thousands                     |
| Loan_Amount_Term | Term of loan in months                       |
| Credit_History   | Credit history meets guidelines              |
| Property_Area    | Urban/ Semi Urban/ Rural                     |
| Loan_Status      | Loan approved (Y/N)                          |

In [ ]:
# Print data types for each variable 
train.dtypes

We can see there are three format of data types:
<ul>
<li><b>object</b>: Object format means variables are categorical. Categorical variables in our dataset are: Loan_ID, Gender, Married, Dependents, Education, Self_Employed, Property_Area, Loan_Status </li>
<li><b>int64</b>: It represents the integer variables. ApplicantIncome is of this format.</li>
<li><b>float64</b>: It represents the variable which have some decimal values involved. They are also numerical variables. Numerical variables in our dataset are: CoapplicantIncome, LoanAmount, Loan_Amount_Term, and Credit_History</li>
</ul>

Let’s look at the shape of the dataset.

In [ ]:
train.shape,test.shape

We have 614 rows and 13 columns in the train dataset and 367 rows and 12 columns in test dataset.

In this section, we will do univariate analysis. It is the simplest form of analyzing data where we examine each variable individually. For categorical features we can use frequency table or bar plots which will calculate the number of each category in a particular variable. For numerical features, probability density plots can be used to look at the distribution of the variable.

### Target Variable
We will first look at the target variable, i.e., Loan_Status. As it is a categorical variable, let us look at its frequency table, percentage distribution and bar plot.
Frequency table of a variable will give us the count of each category in that variable.

In [ ]:
train['Loan_Status'].value_counts()

In [ ]:
# Normalize can be set to True to print proportions instead of number 
train['Loan_Status'].value_counts(normalize=True)

In [ ]:
train['Loan_Status'].value_counts().plot.bar()

The loan of 422(around 69%) people out of 614 was approved.

Now lets visualize each variable separately. Different types of variables are Categorical, ordinal and numerical.
<ul>
<li><b>Categorical features</b>: These features have categories (Gender, Married, Self_Employed, Credit_History,Loan_Status)</li>
<li><b>Ordinal features</b>: Variables in categorical features having some order involved (Dependents, Education, Property_Area)</li>
<li><b>Numerical features</b>: These features have numerical values (ApplicantIncome, CoapplicantIncome, LoanAmount, Loan_Amount_Term)</li>
</ul>

Let’s visualize the categorical and ordinal features first.
### Independent Variable (Categorical)

In [ ]:
plt.figure(1)
plt.subplot(221)
train['Gender'].value_counts(normalize=True).plot.bar(figsize=(20,10), title= 'Gender') 
plt.subplot(222) 
train['Married'].value_counts(normalize=True).plot.bar(title= 'Married') 
plt.subplot(223) 
train['Self_Employed'].value_counts(normalize=True).plot.bar(title= 'Self_Employed') 
plt.subplot(224) 
train['Credit_History'].value_counts(normalize=True).plot.bar(title= 'Credit_History') 
plt.show()

It can be inferred from the above bar plots that:
<ul>
<li>80% applicants in the dataset are male.</li>
<li>Around 65% of the applicants in the dataset are married.</li>
<li>Around 15% applicants in the dataset are self employed.</li>
<li>Around 85% applicants have repaid their debts.</li>
</ul>

Now let’s visualize the ordinal variables.
### Independent Variable (Ordinal)

In [ ]:
plt.figure(1) 
plt.subplot(131) 
train['Dependents'].value_counts(normalize=True).plot.bar(figsize=(24,6), title= 'Dependents') 
plt.subplot(132) 
train['Education'].value_counts(normalize=True).plot.bar(title= 'Education') 
plt.subplot(133) 
train['Property_Area'].value_counts(normalize=True).plot.bar(title= 'Property_Area') 
plt.show()


Following inferences can be made from the above bar plots:
<ul>
<li>Most of the applicants don’t have any dependents.</li>
<li>Around 80% of the applicants are Graduate.</li>
<li>Most of the applicants are from Semiurban area.</li>
</ul>

### Independent Variable (Numerical)
Till now we have seen the categorical and ordinal variables and now lets visualize the numerical variables. Lets look at the distribution of Applicant income first.

In [ ]:
plt.figure(1) 
plt.subplot(121) 
sns.distplot(train['ApplicantIncome']); 
plt.subplot(122) 
train['ApplicantIncome'].plot.box(figsize=(16,5)) 
plt.show()

It can be inferred that most of the data in the distribution of applicant income is towards left which means it is not normally distributed. We will try to make it normal in later sections as algorithms works better if the data is normally distributed.

The boxplot confirms the presence of a lot of outliers/extreme values. This can be attributed to the income disparity in the society. Part of this can be driven by the fact that we are looking at people with different education levels. Let us segregate them by Education:

In [ ]:
train.boxplot(column='ApplicantIncome', by = 'Education') 
plt.suptitle("")

We can see that there are a higher number of graduates with very high incomes, which are appearing to be the outliers.

Let’s look at the Coapplicant income distribution.

In [ ]:
plt.figure(1) 
plt.subplot(121) 
sns.distplot(train['CoapplicantIncome']); 
plt.subplot(122) 
train['CoapplicantIncome'].plot.box(figsize=(16,5)) 
plt.show()

We see a similar distribution as that of the applicant income. Majority of coapplicant’s income ranges from 0 to 5000. We also see a lot of outliers in the coapplicant income and it is not normally distributed.

Let’s look at the distribution of LoanAmount variable.

In [ ]:
plt.figure(1) 
plt.subplot(121) 
df=train.dropna() 
sns.distplot(df['LoanAmount']); 
plt.subplot(122) 
train['LoanAmount'].plot.box(figsize=(16,5)) 
plt.show()

We see a lot of outliers in this variable and the distribution is fairly normal. We will treat the outliers in later sections.

Now we would like to know how well each feature correlate with Loan Status. So, in the next section we will look at bivariate analysis.

Lets recall some of the hypotheses that we generated earlier:
<ul>
<li>Applicants with high income should have more chances of loan approval.</li>
<li>Applicants who have repaid their previous debts should have higher chances of loan approval.</li>
<li>Loan approval should also depend on the loan amount. If the loan amount is less, chances of loan approval should be high.</li>
<li>Lesser the amount to be paid monthly to repay the loan, higher the chances of loan approval.</li>
</ul>

Lets try to test the above mentioned hypotheses using bivariate analysis

After looking at every variable individually in univariate analysis, we will now explore them again with respect to the target variable.
### Categorical Independent Variable vs Target Variable

First of all we will find the relation between target variable and categorical independent variables. Let us look at the stacked bar plot now which will give us the proportion of approved and unapproved loans.

In [ ]:
Gender=pd.crosstab(train['Gender'],train['Loan_Status']) 
Gender.div(Gender.sum(1).astype(float), axis=0).plot(kind="bar", stacked=True, figsize=(4,4))

It can be inferred that the proportion of male and female applicants is more or less same for both approved and unapproved loans.

Now let us visualize the remaining categorical variables vs target variable.

In [ ]:
Married=pd.crosstab(train['Married'],train['Loan_Status']) 
Dependents=pd.crosstab(train['Dependents'],train['Loan_Status']) 
Education=pd.crosstab(train['Education'],train['Loan_Status']) 
Self_Employed=pd.crosstab(train['Self_Employed'],train['Loan_Status']) 
Married.div(Married.sum(1).astype(float), axis=0).plot(kind="bar", stacked=True, figsize=(4,4)) 
plt.show() 
Dependents.div(Dependents.sum(1).astype(float), axis=0).plot(kind="bar", stacked=True) 
plt.show() 
Education.div(Education.sum(1).astype(float), axis=0).plot(kind="bar", stacked=True, figsize=(4,4)) 
plt.show() 
Self_Employed.div(Self_Employed.sum(1).astype(float), axis=0).plot(kind="bar", stacked=True, figsize=(4,4)) 
plt.show()

<ul>
<li>Proportion of married applicants is higher for the approved loans.</li>
<li>Distribution of applicants with 1 or 3+ dependents is similar across both the categories of Loan_Status.</li>
<li>There is nothing significant we can infer from Self_Employed vs Loan_Status plot.</li>
</ul>

Now we will look at the relationship between remaining categorical independent variables and Loan_Status.

In [ ]:
Credit_History=pd.crosstab(train['Credit_History'],train['Loan_Status']) 
Property_Area=pd.crosstab(train['Property_Area'],train['Loan_Status']) 
Credit_History.div(Credit_History.sum(1).astype(float), axis=0).plot(kind="bar", stacked=True, figsize=(4,4)) 
plt.show() 
Property_Area.div(Property_Area.sum(1).astype(float), axis=0).plot(kind="bar", stacked=True) 
plt.show()

<ul>
<li>It seems people with credit history as 1 are more likely to get their loans approved.</li>
<li>Proportion of loans getting approved in semiurban area is higher as compared to that in rural or urban areas.</li>
</ul>

Now let’s visualize numerical independent variables with respect to target variable.
### Numerical Independent Variable vs Target Variable
We will try to find the mean income of people for which the loan has been approved vs the mean income of people for which the loan has not been approved.

In [ ]:
train.groupby('Loan_Status')['ApplicantIncome'].mean().plot.bar()

Here the y-axis represents the mean applicant income. We don’t see any change in the mean income. So, let’s make bins for the applicant income variable based on the values in it and analyze the corresponding loan status for each bin.

In [ ]:
bins=[0,2500,4000,6000,81000] 
group=['Low','Average','High', 'Very high'] 
train['Income_bin']=pd.cut(df['ApplicantIncome'],bins,labels=group)
Income_bin=pd.crosstab(train['Income_bin'],train['Loan_Status']) 
Income_bin.div(Income_bin.sum(1).astype(float), axis=0).plot(kind="bar", stacked=True) 
plt.xlabel('ApplicantIncome') 
P = plt.ylabel('Percentage')

It can be inferred that Applicant income does not affect the chances of loan approval which contradicts our hypothesis in which we assumed that if the applicant income is high the chances of loan approval will also be high.

We will analyze the coapplicant income and loan amount variable in similar manner.

In [ ]:
bins=[0,1000,3000,42000] 
group=['Low','Average','High'] 
train['Coapplicant_Income_bin']=pd.cut(df['CoapplicantIncome'],bins,labels=group)

In [ ]:
Coapplicant_Income_bin=pd.crosstab(train['Coapplicant_Income_bin'],train['Loan_Status']) 
Coapplicant_Income_bin.div(Coapplicant_Income_bin.sum(1).astype(float), axis=0).plot(kind="bar", stacked=True) 
plt.xlabel('CoapplicantIncome') 
P = plt.ylabel('Percentage')

It shows that if coapplicant’s income is less the chances of loan approval are high. But this does not look right. The possible reason behind this may be that most of the applicants don’t have any coapplicant so the coapplicant income for such applicants is 0 and hence the loan approval is not dependent on it. So we can make a new variable in which we will combine the applicant’s and coapplicant’s income to visualize the combined effect of income on loan approval.

Let us combine the Applicant Income and Coapplicant Income and see the combined effect of Total Income on the Loan_Status.

In [ ]:
train['Total_Income']=train['ApplicantIncome']+train['CoapplicantIncome']
bins=[0,2500,4000,6000,81000] 
group=['Low','Average','High', 'Very high'] 
train['Total_Income_bin']=pd.cut(train['Total_Income'],bins,labels=group)
Total_Income_bin=pd.crosstab(train['Total_Income_bin'],train['Loan_Status'])
Total_Income_bin.div(Total_Income_bin.sum(1).astype(float), axis=0).plot(kind="bar", stacked=True)
plt.xlabel('Total_Income') 
P = plt.ylabel('Percentage')


We can see that Proportion of loans getting approved for applicants having low Total_Income is very less as compared to that of applicants with Average, High and Very High Income.

Let’s visualize the Loan amount variable.

In [ ]:
bins=[0,100,200,700] 
group=['Low','Average','High'] 
train['LoanAmount_bin']=pd.cut(df['LoanAmount'],bins,labels=group)
LoanAmount_bin=pd.crosstab(train['LoanAmount_bin'],train['Loan_Status']) 
LoanAmount_bin.div(LoanAmount_bin.sum(1).astype(float), axis=0).plot(kind="bar", stacked=True) 
plt.xlabel('LoanAmount') 
P = plt.ylabel('Percentage')

It can be seen that the proportion of approved loans is higher for Low and Average Loan Amount as compared to that of High Loan Amount which supports our hypothesis in which we considered that the chances of loan approval will be high when the loan amount is less.

Let’s drop the bins which we created for the exploration part. We will change the 3+ in dependents variable to 3 to make it a numerical variable.We will also convert the target variable’s categories into 0 and 1 so that we can find its correlation with numerical variables. One more reason to do so is few models like logistic regression takes only numeric values as input. We will replace N with 0 and Y with 1.

In [ ]:
train=train.drop(['Income_bin', 'Coapplicant_Income_bin', 'LoanAmount_bin', 'Total_Income_bin', 'Total_Income'], axis=1)
train['Dependents'].replace('3+', 3,inplace=True) 
test['Dependents'].replace('3+', 3,inplace=True) 
train['Loan_Status'].replace('N', 0,inplace=True) 
train['Loan_Status'].replace('Y', 1,inplace=True)

Now lets look at the correlation between all the numerical variables. We will use the heat map to visualize the correlation. Heatmaps visualize data through variations in coloring. The variables with darker color means their correlation is more.

In [ ]:
matrix = train.corr() 
f, ax = plt.subplots(figsize=(9, 6)) 
sns.heatmap(matrix, vmax=.8, square=True, cmap="BuPu");

We see that the most correlated variables are (ApplicantIncome - LoanAmount) and (Credit_History - Loan_Status). LoanAmount is also correlated with CoapplicantIncome.

After exploring all the variables in our data, we can now impute the missing values and treat the outliers because missing data and outliers can have adverse effect on the model performance.
### Missing value imputation
Let’s list out feature-wise count of missing values.

In [ ]:
train.isnull().sum()

There are missing values in Gender, Married, Dependents, Self_Employed, LoanAmount, Loan_Amount_Term and Credit_History features.

We will treat the missing values in all the features one by one.

We can consider these methods to fill the missing values:

<ul>
<li><b>For numerical variables</b>: imputation using mean or median</li>
<li><b>For categorical variables</b>: imputation using mode</li>
</ul>

There are very less missing values in Gender, Married, Dependents, Credit_History and Self_Employed features so we can fill them using the mode of the features.

In [ ]:
train['Gender'].fillna(train['Gender'].mode()[0], inplace=True) 
train['Married'].fillna(train['Married'].mode()[0], inplace=True) 
train['Dependents'].fillna(train['Dependents'].mode()[0], inplace=True) 
train['Self_Employed'].fillna(train['Self_Employed'].mode()[0], inplace=True) 
train['Credit_History'].fillna(train['Credit_History'].mode()[0], inplace=True)


Now let’s try to find a way to fill the missing values in Loan_Amount_Term. We will look at the value count of the Loan amount term variable.

In [ ]:
train['Loan_Amount_Term'].value_counts()

It can be seen that in loan amount term variable, the value of 360 is repeating the most. So we will replace the missing values in this variable using the mode of this variable.

In [ ]:
train['Loan_Amount_Term'].fillna(train['Loan_Amount_Term'].mode()[0], inplace=True)


Now we will see the LoanAmount variable. As it is a numerical variable, we can use mean or median to impute the missing values. We will use median to fill the null values as earlier we saw that loan amount have outliers so the mean will not be the proper approach as it is highly affected by the presence of outliers.

In [ ]:
train['LoanAmount'].fillna(train['LoanAmount'].median(), inplace=True)

Now lets check whether all the missing values are filled in the dataset.

In [ ]:
train.isnull().sum()

As we can see that all the missing values have been filled in the train dataset. Let’s fill all the missing values in the test dataset too with the same approach.

In [ ]:
test['Gender'].fillna(train['Gender'].mode()[0], inplace=True) 
test['Dependents'].fillna(train['Dependents'].mode()[0], inplace=True) 
test['Self_Employed'].fillna(train['Self_Employed'].mode()[0], inplace=True) 
test['Credit_History'].fillna(train['Credit_History'].mode()[0], inplace=True) 
test['Loan_Amount_Term'].fillna(train['Loan_Amount_Term'].mode()[0], inplace=True) 
test['LoanAmount'].fillna(train['LoanAmount'].median(), inplace=True)

### Outlier Treatment

As we saw earlier in univariate analysis, LoanAmount contains outliers so we have to treat them as the presence of outliers affects the distribution of the data. Let's examine what can happen to a data set with outliers. For the sample data set:

1, 1, 2, 2, 2, 2, 3, 3, 3, 4, 4

We find the following: mean, median, mode, and standard deviation

Mean = 2.58

Median = 2.5

Mode = 2

Standard Deviation = 1.08

If we add an outlier to the data set:

1, 1, 2, 2, 2, 2, 3, 3, 3, 4, 4, 400

The new values of our statistics are:

Mean = 35.38

Median = 2.5

Mode = 2

Standard Deviation = 114.74

It can be seen that having outliers often has a significant effect on the mean and standard deviation and hence affecting the distribution. We must take steps to remove outliers from our data sets.

Due to these outliers bulk of the data in the loan amount is at the left and the right tail is longer. This is called right skewness. One way to remove the skewness is by doing the log transformation. As we take the log transformation, it does not affect the smaller values much, but reduces the larger values. So, we get a distribution similar to normal distribution.

Let’s visualize the effect of log transformation. We will do the similar changes to the test file simultaneously.

In [ ]:
train['LoanAmount_log'] = np.log(train['LoanAmount']) 
train['LoanAmount_log'].hist(bins=20) 
test['LoanAmount_log'] = np.log(test['LoanAmount'])

Now the distribution looks much closer to normal and effect of extreme values has been significantly subsided. Let’s build a logistic regression model and make predictions for the test dataset.

The process of model building is not complete without evaluation of model’s performance. Suppose we have the predictions from the model, how can we decide whether the predictions are accurate? We can plot the results and compare them with the actual values, i.e. calculate the distance between the predictions and actual values. Lesser this distance more accurate will be the predictions. Since this is a classification problem, we can evaluate our models using any one of the following evaluation metrics:

<ul><li><b>Accuracy</b>: Let us understand it using the confusion matrix which is a tabular representation of Actual vs Predicted values. This is how a confusion matrix looks like:</li><ul>

<img src = 'https://2.bp.blogspot.com/-EvSXDotTOwc/XMfeOGZ-CVI/AAAAAAAAEiE/oePFfvhfOQM11dgRn9FkPxlegCXbgOF4QCLcBGAs/s1600/confusionMatrxiUpdated.jpg'>

True Positive - Targets which are actually true(Y) and we have predicted them true(Y)
True Negative - Targets which are actually false(N) and we have predicted them false(N)
False Positive - Targets which are actually false(N) but we have predicted them true(T)
False Negative - Targets which are actually true(T) but we have predicted them false(N)
<ul>
<li><b>True Positive</b> - Targets which are actually true(Y) and we have predicted them true(Y)</li>
<li><b>True Negative </b>- Targets which are actually false(N) and we have predicted them false(N)</li>
<li><b>False Positive</b> - Targets which are actually false(N) but we have predicted them true(T)</li>
<li><b>False Negative</b> - Targets which are actually true(T) but we have predicted them false(N)</li>
</ul>


Using these values, we can calculate the accuracy of the model. The accuracy is given by:

<img src  = 'https://media.geeksforgeeks.org/wp-content/uploads/Confusion_Matrix2_2.png'>

<ul><li><b>Precision</b>: It is a measure of correctness achieved in true prediction i.e. of observations labeled as true, how many are actually labeled true.</li></ul>

Precision = TP / (TP + FP)

<ul><li><b>Recall(Sensitivity)</b>: It is a measure of actual observations which are predicted correctly i.e. how many observations of true class are labeled correctly. It is also known as ‘Sensitivity’.</li></ul>

Recall = TP / (TP + FN)

<ul><li><b>Specificity</b>: It is a measure of how many observations of false class are labeled correctly.</li></ul>

Specificity = TN / (TN + FP)


Specificity and Sensitivity plays a crucial role in deriving ROC curve.




<ul><li>ROC curve</li>
<li>Receiver Operating Characteristic(ROC) summarizes the model’s performance by evaluating the trade offs between true positive rate (sensitivity) and false positive rate(1- specificity).</li>
<li>The area under curve (AUC), referred to as index of accuracy(A) or concordance index, is a perfect performance metric for ROC curve. Higher the area under curve, better the prediction power of the model.</li></ul>

This is how a ROC curve looks like:

<img src = 'https://i.stack.imgur.com/yXkG7.png'>

<ul>
<li>The area of this curve measures the ability of the model to correctly classify true positives and true negatives. We want our model to predict the true classes as true and false classes as false.</li>
<li>So it can be said that we want the true positive rate to be 1. But we are not concerned with the true positive rate only but the false positive rate too. For example in our problem, we are not only concerned about predicting the Y classes as Y but we also want N classes to be predicted as N.</li>
<li>We want to increase the area of the curve which will be maximum for class 2,3,4 and 5 in the above example.</li>
<li>For class 1 when the false positive rate is 0.2, the true positive rate is around 0.6. But for class 2 the true positive rate is 1 at the same false positive rate. So, the AUC for class 2 will be much more as compared to the AUC for class 1. So, the model for class 2 will be better.</li>
<li>The class 2,3,4 and 5 model will predict more accurately as compared to the class 0 and 1 model as the AUC is more for those classes.</li>
</ul>

Let us make our first model to predict the target variable. We will start with Logistic Regression which is used for predicting binary outcome.

<ul><li>Logistic Regression is a classification algorithm. It is used to predict a binary outcome (1 / 0, Yes / No, True / False) given a set of independent variables.</li>
<li>Logistic regression is an estimation of Logit function. Logit function is simply a log of odds in favor of the event.</li>
<li>This function creates a s-shaped curve with the probability estimate, which is very similar to the required step wise function</li></ul>

Lets drop the Loan_ID variable as it do not have any effect on the loan status. We will do the same changes to the test dataset which we did for the training dataset.

In [ ]:
train=train.drop('Loan_ID',axis=1) 
test=test.drop('Loan_ID',axis=1)

We will use scikit-learn (sklearn) for making different models which is an open source library for Python. It is one of the most efficient tool which contains many inbuilt functions that can be used for modeling in Python.

Sklearn requires the target variable in a separate dataset. So, we will drop our target variable from the train dataset and save it in another dataset.

In [ ]:
X = train.drop('Loan_Status',1)
y = train.Loan_Status

Now we will make dummy variables for the categorical variables. Dummy variable turns categorical variables into a series of 0 and 1, making them lot easier to quantify and compare. Let us understand the process of dummies first:

<ul>
<li>Consider the “Gender” variable. It has two classes, Male and Female.</li>
<li>As logistic regression takes only the numerical values as input, we have to change male and female into numerical value</li>
<li>We want to increase the area of the curve which will be maximum for class 2,3,4 and 5 in the above example.</li>
<li>Once we apply dummies to this variable, it will convert the “Gender” variable into two variables(Gender_Male and Gender_Female), one for each class, i.e. Male and Female.</li>
<li>Gender_Male will have a value of 0 if the gender is Female and a value of 1 if the gender is Male.</li>

</ul>

In [ ]:
X=pd.get_dummies(X) 
train=pd.get_dummies(train) 
test=pd.get_dummies(test)

Now we will train the model on training dataset and make predictions for the test dataset. But can we validate these predictions? One way of doing this is we can divide our train dataset into two parts: train and validation. We can train the model on this train part and using that make predictions for the validation part. In this way we can validate our predictions as we have the true predictions for the validation part (which we do not have for the test dataset).

We will use the train_test_split function from sklearn to divide our train dataset. So, first let us import train_test_split.

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_cv, y_train, y_cv = train_test_split(X,y, test_size =0.3)

The dataset has been divided into training and validation part. Let us import LogisticRegression and accuracy_score from sklearn and fit the logistic regression model.



In [ ]:
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import accuracy_score
model = LogisticRegression() 
model.fit(x_train, y_train)

Here the C parameter represents inverse of regularization strength. Regularization is applying a penalty to increasing the magnitude of parameter values in order to reduce overfitting. Smaller values of C specify stronger regularization. To learn about other parameters, [refer here:](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)


Let’s predict the Loan_Status for validation set and calculate its accuracy.

In [ ]:
pred_cv = model.predict(x_cv)
accuracy_score(y_cv,pred_cv)

Let’s make predictions for the test dataset.

In [ ]:
pred_test = model.predict(test)

To check how robust our model is to unseen data, we can use Validation. It is a technique which involves reserving a particular sample of a dataset on which you do not train the model. Later, you test your model on this sample before finalizing it. Some of the common methods for validation are listed below:

<ul>
<li>The validation set approach</li>
<li>k-fold cross validation</li>
<li>We want to increase the area of the curve which will be maximum for class 2,3,4 and 5 in the above example.</li>
<li>Leave one out cross validation (LOOCV)</li>
<li>Stratified k-fold cross validation</li>
</ul>

In this section we will learn about stratified k-fold cross validation. Let us understand how it works:

<ul>
<li>Stratification is the process of rearranging the data so as to ensure that each fold is a good representative of the whole.</li>
<li>For example, in a binary classification problem where each class comprises of 50% of the data, it is best to arrange the data such that in every fold, each class comprises of about half the instances.</li>
<li>It is generally a better approach when dealing with both bias and variance.</li>
<li>A randomly selected fold might not adequately represent the minor class, particularly in cases where there is a huge class imbalance.</li>
</ul>

Below is the visualization of a stratified k-fold validation when k=5.
<img src = 'https://s3-ap-south-1.amazonaws.com/av-blog-media/wp-content/uploads/2015/11/skfold-768x530.png'>

Let’s import StratifiedKFold from sklearn and fit the model.

In [ ]:
from sklearn.model_selection import StratifiedKFold

Now let’s make a cross validation logistic model with stratified 5 folds and make predictions for test dataset.

In [ ]:
i=1 
kf = StratifiedKFold(n_splits=5,random_state=1,shuffle=True) 
for train_index,test_index in kf.split(X,y):     
    print('\n{} of kfold {}'.format(i,kf.n_splits))     
    xtr,xvl = X.loc[train_index],X.loc[test_index]     
    ytr,yvl = y[train_index],y[test_index]         
    model = LogisticRegression(random_state=1)     
    model.fit(xtr, ytr)     
    pred_test = model.predict(xvl)     
    score = accuracy_score(yvl,pred_test)     
    print('accuracy_score',score)     
    i+=1 
pred_test = model.predict(test) 
pred=model.predict_proba(xvl)[:,1]

The mean validation accuracy for this model turns out to be 0.83. Let us visualize the roc curve.

In [ ]:
from sklearn import metrics 
fpr, tpr, _ = metrics.roc_curve(yvl,  pred) 
auc = metrics.roc_auc_score(yvl, pred) 
plt.figure(figsize=(12,8)) 
plt.plot(fpr,tpr,label="validation, auc="+str(auc)) 
plt.xlabel('False Positive Rate') 
plt.ylabel('True Positive Rate') 
plt.legend(loc=4) 
plt.show()

Based on the domain knowledge, we can come up with new features that might affect the target variable. We will create the following three new features:

<ul>
<li><b>Total Income</b> - As discussed during bivariate analysis we will combine the Applicant Income and Coapplicant Income. If the total income is high, chances of loan approval might also be high.</li>
<li><b>EMI</b> - EMI is the monthly amount to be paid by the applicant to repay the loan. Idea behind making this variable is that people who have high EMI’s might find it difficult to pay back the loan. We can calculate the EMI by taking the ratio of loan amount with respect to loan amount term.</li>
<li><b>Balance Income</b> - This is the income left after the EMI has been paid. Idea behind creating this variable is that if this value is high, the chances are high that a person will repay the loan and hence increasing the chances of loan approval.</li>
</ul>

In [ ]:
train['Total_Income']=train['ApplicantIncome']+train['CoapplicantIncome'] 
test['Total_Income']=test['ApplicantIncome']+test['CoapplicantIncome']

Let’s check the distribution of Total Income.

In [ ]:
sns.distplot(train['Total_Income']);

We can see it is shifted towards left, i.e., the distribution is right skewed. So, let’s take the log transformation to make the distribution normal.

In [ ]:
train['Total_Income_log'] = np.log(train['Total_Income']) 
sns.distplot(train['Total_Income_log']); 
test['Total_Income_log'] = np.log(test['Total_Income'])

Now the distribution looks much closer to normal and effect of extreme values has been significantly subsided. Let’s create the EMI feature now.

In [ ]:
train['EMI']=train['LoanAmount']/train['Loan_Amount_Term'] 
test['EMI']=test['LoanAmount']/test['Loan_Amount_Term']

Let’s check the distribution of EMI variable.

In [ ]:
sns.distplot(train['EMI']);

Let us create Balance Income feature now and check its distribution.

In [ ]:
train['Balance Income']=train['Total_Income']-(train['EMI']*1000) # Multiply with 1000 to make the units equal test['Balance Income']=test['Total_Income']-(test['EMI']*1000)
test['Balance Income']=test['Total_Income']-(test['EMI']*1000)
sns.distplot(train['Balance Income']);


Let us now drop the variables which we used to create these new features. Reason for doing this is, the correlation between those old features and these new features will be very high and logistic regression assumes that the variables are not highly correlated. We also wants to remove the noise from the dataset, so removing correlated features will help in reducing the noise too.

In [ ]:
train=train.drop(['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term'], axis=1) 
test=test.drop(['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term'], axis=1)

After creating new features, we can continue the model building process. So we will start with logistic regression model and then move over to more complex models like RandomForest and XGBoost.

We will build the following models in this section.

<ul>
<li>Random Forest</li>
<li>Logistic Regression</li>
<li>Decision Tree</li>
<li>XGBoost</li>
</ul>

Let’s prepare the data for feeding into the models.

In [ ]:
X = train.drop('Loan_Status',1) 
y = train.Loan_Status

In [ ]:
from statistics import mean

### Logistic Regression

In [ ]:
i=1 
score_list = []
kf = StratifiedKFold(n_splits=5,random_state=1,shuffle=True) 
for train_index,test_index in kf.split(X,y):     
    print('\n{} of kfold {}'.format(i,kf.n_splits))     
    xtr,xvl = X.loc[train_index],X.loc[test_index]     
    ytr,yvl = y[train_index],y[test_index]         

    model = LogisticRegression(random_state=1)     
    model.fit(xtr, ytr)     
    pred_test = model.predict(xvl)     
    score = accuracy_score(yvl,pred_test)     
    print('accuracy_score',score)     
    score_list.append(score)
    i+=1 
pred_test = model.predict(test) 
pred=model.predict_proba(xvl)[:,1]

log_reg = round(model.score(xtr,ytr) * 100, 2)
cv1 = round(mean(score_list)*100,2)

print()
print('Training Score',log_reg)
print('Mean Cross-validation Score:', cv1)

### Decision Tree

In [ ]:
from sklearn import tree

In [ ]:
i=1 
score_list = []
kf = StratifiedKFold(n_splits=5,random_state=1,shuffle=True) 
for train_index,test_index in kf.split(X,y):     
    print('\n{} of kfold {}'.format(i,kf.n_splits))     
    xtr,xvl = X.loc[train_index],X.loc[test_index]     
    ytr,yvl = y[train_index],y[test_index]         
    model = tree.DecisionTreeClassifier(random_state=1)     
    model.fit(xtr, ytr)     
    pred_test = model.predict(xvl)     
    score = accuracy_score(yvl,pred_test)     
    print('accuracy_score',score)     
    score_list.append(score)
    i+=1 
pred_test = model.predict(test)

d_tree = round(model.score(xtr,ytr) * 100, 2)
cv2 = round(mean(score_list)*100,2)

print()
print('Training Score',d_tree)
print('Mean Cross-validation Score:', cv2)

### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
i=1 
score_list =[]
kf = StratifiedKFold(n_splits=5,random_state=1,shuffle=True) 
for train_index,test_index in kf.split(X,y):     
    print('\n{} of kfold {}'.format(i,kf.n_splits))     
    xtr,xvl = X.loc[train_index],X.loc[test_index]     
    ytr,yvl = y[train_index],y[test_index]         
    model = RandomForestClassifier(random_state=1, max_depth=10)     
    model.fit(xtr, ytr)     
    pred_test = model.predict(xvl)     
    score = accuracy_score(yvl,pred_test)     
    print('accuracy_score',score)     
    score_list.append(score)
    i+=1 

pred_test = model.predict(test)
r_forest = round(model.score(xtr,ytr) * 100, 2)
cv3 = round(mean(score_list)*100,2)
print()
print('Training Score',r_forest)
print('Mean Cross-validation Score:', cv3)

We will try to improve the accuracy by tuning the hyperparameters for this model. We will use grid search to get the optimized values of hyper parameters. Grid-search is a way to select the best of a family of hyper parameters, parametrized by a grid of parameters.

We will tune the max_depth and n_estimators parameters. max_depth decides the maximum depth of the tree and n_estimators decides the number of trees that will be used in random forest model.

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
# Provide range for max_depth from 1 to 20 with an interval of 2 and from 1 to 200 with an interval of 20 for n_estimators 
paramgrid = {'max_depth': list(range(1, 20, 2)), 'n_estimators': list(range(1, 200, 20))}

In [ ]:
grid_search=GridSearchCV(RandomForestClassifier(random_state=1),paramgrid)

In [ ]:
from sklearn.model_selection import train_test_split 
x_train, x_cv, y_train, y_cv = train_test_split(X,y, test_size =0.3, random_state=1)

In [ ]:
# Fit the grid search model 
grid_search.fit(x_train,y_train)

In [ ]:
# Estimating the optimized value 
grid_search.best_estimator_

So, the optimized value for the max_depth variable is 3 and for n_estimator is 41. Now let’s build the model using these optimized values.

In [ ]:
i=1 
score_list=[]
kf = StratifiedKFold(n_splits=5,random_state=1,shuffle=True) 
for train_index,test_index in kf.split(X,y):     
    print('\n{} of kfold {}'.format(i,kf.n_splits))     
    xtr,xvl = X.loc[train_index],X.loc[test_index]     
    ytr,yvl = y[train_index],y[test_index]         
    model = RandomForestClassifier(random_state=1, max_depth=3, n_estimators=41)     
    model.fit(xtr, ytr)     
    pred_test = model.predict(xvl)     
    score = accuracy_score(yvl,pred_test)     
    print('accuracy_score',score)     
    score_list.append(score)
    i+=1 
pred_test = model.predict(test) 
pred2=model.predict_proba(test)[:,1]
r_forest_fine_tuned = round(model.score(xtr,ytr) * 100, 2)
cv4 = round(mean(score_list)*100,2)
print()
print('Training Score: ',r_forest_fine_tuned)
print('Mean Cross-validation Score: ', cv4)

### XGBOOST

XGBoost is a fast and efficient algorithm and has been used to by the winners of many data science competitions. It’s a boosting algorithm and you may refer the below article to know more about boosting:<a> https://www.analyticsvidhya.com/blog/2015/11/quick-introduction-boosting-algorithms-machine-learning/</a>
XGBoost works only with numeric variables and we have already replaced the categorical variables with numeric variables. Let’s have a look at the parameters that we are going to use in our model.

<ul><li>n_estimator: This specifies the number of trees for the model.</li>
<li>max_depth: We can specify maximum depth of a tree using this parameter.</li></ul>

In [ ]:
from xgboost import XGBClassifier

In [ ]:
i=1
score_list=[]
kf = StratifiedKFold(n_splits=5,random_state=1,shuffle=True) 
for train_index,test_index in kf.split(X,y):     
    print('\n{} of kfold {}'.format(i,kf.n_splits))     
    xtr,xvl = X.loc[train_index],X.loc[test_index]     
    ytr,yvl = y[train_index],y[test_index]         
    model = XGBClassifier(n_estimators=50, max_depth=4)     
    model.fit(xtr, ytr)     
    pred_test = model.predict(xvl)     
    score = accuracy_score(yvl,pred_test)     
    print('accuracy_score',score)
    score_list.append(score)
    i+=1 
pred_test = model.predict(test) 
pred3=model.predict_proba(test)[:,1]
xg_boost = round(model.score(xtr,ytr) * 100, 2)
cv5 = round(mean(score_list)*100,2)
print()
print('Training Score',xg_boost)
print('Mean Cross-validation Score:', cv5)

In [ ]:
score_table = pd.DataFrame([log_reg,d_tree,r_forest,r_forest_fine_tuned,xg_boost],columns=['Training Score'])
score_table['Mean Cross Validation Score'] = [cv1,cv2,cv3,cv4,cv5]
score_table['Algorithms'] = ['Logistic Regression','Decision Tree','Random Forest','Random Forest Fine tuned','XG Boost']
score_table.set_index('Algorithms')